[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juanipis/AI-path-finder/blob/main/path_map_gradio.ipynb)

In [ ]:
# Va a pedir reconexión de entorno de ejecución, es necesario para que los cambios tengan efecto.
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/Juanipis/AI-path-finder.git
    !mv AI-path-finder/* ./
    #!rm -r AI-path-finder
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !conda install gdal rasterio osmnx
    %pip install -r requirements.txt
    print("Repositorio clonado con éxito, recargue el entorno de ejcución.")
else:
    print("No estás en Google Colab. Asegúrate de clonar el repositorio manualmente y tener las librerias instaladas.")

In [ ]:
import gradio as gr
from utils.map2 import Map

class MapGradioInterface:
    def __init__(self):
        self.map = None

    def initialize_map(self, location, simplify):
        self.map = Map(location=location, simplify=simplify)
        self.map.set_data()
        return "Map initialized successfully!"

    def calculate_route(self, start, end, algorithm):
        if self.map is None:
            return "Please initialize the map first!"

        try:
            start_node, end_node = self.map.set_route(start, end)
            if algorithm == "shortest_path":
                route = self.map.shortest_route(start_node, end_node)
            else:
                route = self.map.astar_route(start_node, end_node)
            fig = self.map.display_interactive_route(route, start_node, end_node)
            return fig
        except Exception as e:
            return f"An error occurred: {str(e)}"

    def launch(self):
        with gr.Blocks() as demo:
            gr.Markdown("# Map Route Calculator")

            with gr.Tab("Initialize Map"):
                location_input = gr.Textbox(label="Location", value="Envigado, Antioquia, Colombia")
                simplify_checkbox = gr.Checkbox(label="Simplify", value=False)
                init_button = gr.Button("Initialize Map")
                init_output = gr.Textbox(label="Initialization Status")

                init_button.click(
                    self.initialize_map,
                    inputs=[location_input, simplify_checkbox],
                    outputs=init_output
                )

            with gr.Tab("Calculate Route"):
                start_input = gr.Textbox(label="Start Location", value="Sede Posgrados EIA")
                end_input = gr.Textbox(label="End Location", value="Universidad EIA, Envigado, Colombia")
                algorithm_dropdown = gr.Dropdown(
                    choices=["shortest_path", "a_star"], 
                    label="Algorithm", 
                    value="shortest_path"
                )
                route_button = gr.Button("Calculate Route")
                route_output = gr.Plot()

                route_button.click(
                    self.calculate_route,
                    inputs=[start_input, end_input, algorithm_dropdown],
                    outputs=route_output
                )

        demo.launch()

if __name__ == "__main__":
    interface = MapGradioInterface()
    interface.launch()